In [245]:
import PyPDF2
import regex as re
from nltk import word_tokenize 
from nltk.util import ngrams
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from dictionary import *
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json
import os
import glob
from tqdm import tqdm
# filename is test.pdf

In [246]:
os.chdir('/home/uttkarsh/IP/')
print(os.getcwd())
def get_text(file):
    text = ''
    with open(file, 'rb') as pdfFileObj:
        pdfReader = PyPDF2.PdfReader(pdfFileObj)
        num_pages = len(pdfReader.pages)
        for i in range(num_pages):
            pageObj = pdfReader.pages[i]
            text += pageObj.extract_text()
        text = text.replace('\n', '')
    return text

def get_string_from_list(list):
    string = ''
    for i in range(len(list)):
        string += list[i]
        if i != len(list)-1:
            string += ' '
    return string

def filter(text):
    token = word_tokenize(text)
    token = [word.lower() for word in token]
    token = [word for word in token if word.isalpha()]
    token = [word for word in token if (word == 'down' or not word in stop_words )]
    # remoev single character
    token = [word for word in token if len(word) > 1]
    # remove th
    token = [word for word in token if word != 'th']
    for i in range(len(token)):
        if token[i] == 'per' and token[i+1] == 'cent':
            token[i] = 'percent'
            token[i+1] = ''
    token = [word for word in token if word != '']
    return token

def get_bigram(token):
    token = [stemmer.stem(word) for word in token]
    bigram = ngrams(token, 2)
    return list(bigram)

def get_trigram(token):
    token = [stemmer.stem(word) for word in token]
    trigram = ngrams(token, 3)
    return list(trigram)

/home/uttkarsh/IP


In [247]:
def searchAndCount_single(bigram_list, term, hdlist):
    term = stemmer.stem(term)
    count = 0
    for i in range(len(bigram_list)):
        if term in bigram_list[i]:
            for j in range(len(hdlist)):
                if i-7 < 0:
                    for k in range(i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                elif i+7 > len(bigram_list):
                    for k in range(i, len(bigram_list)):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                else:
                    for k in range(i-7, i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
    return count


def searchAndCount_double(bigram_list, term1, term2, hdlist):
    term1 = stemmer.stem(term1)
    term2 = stemmer.stem(term2)

    count = 0
    for i in range(len(bigram_list)):
        if term1 in bigram_list[i] and term2 in bigram_list[i]:
            for j in range(len(hdlist)):
                if i-7 < 0:
                    for k in range(i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                elif i+7 > len(bigram_list):
                    for k in range(i, len(bigram_list)):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
                else:
                    for k in range(i-7, i+7):
                        if hdlist[j] in bigram_list[k]:
                            count += 1
    return count

def searchAndCount_triple(trigram_list, term1, term2, term3, hdlist):
    term1 = stemmer.stem(term1)
    term2 = stemmer.stem(term2)
    term3 = stemmer.stem(term3)
    count = 0
    for i in range(len(trigram_list)):
        if term1 in trigram_list[i] and term2 in trigram_list[i] and term3 in trigram_list[i]:
            for j in range(len(hdlist)):
                if i-7 < 0:
                    for k in range(i+7):
                        if hdlist[j] in trigram_list[k]:
                            count += 1
                elif i+7 > len(trigram_list):
                    for k in range(i, len(trigram_list)):
                        if hdlist[j] in trigram_list[k]:
                            count += 1
                else:
                    for k in range(i-7, i+7):
                        if hdlist[j] in trigram_list[k]:
                            count += 1
    return count


In [248]:
cpi_hawkish = searchAndCount_triple(trigram_list, "consumer", "prices", "inflation", ConsumerPricesInflation_Hawkish)
print(cpi_hawkish)
cpi_dovish = searchAndCount_triple(trigram_list,  "consumer", "prices", "inflation", ConsumerPricesInflation_Dovish)
print(cpi_dovish)

ip_hawkish = searchAndCount_single(bigram_list, "inflation", InflationPressure_Hawkish)
print(ip_hawkish)
ip_dovish = searchAndCount_single(bigram_list, "inflation", InflationPressure_Dovish)
print(ip_dovish)

cs_hawkish = searchAndCount_double(bigram_list, "consumer", "spending", ConsumerSpending_Hawkish)
print(cs_hawkish)
cs_dovish = searchAndCount_double(bigram_list, "consumer", "spending", ConsumerSpending_Dovish)
print(cs_dovish)

ea_hawkish = searchAndCount_single(bigram_list, "econom", EconomicActivity_Hawkish)
print(ea_hawkish)
ea_dovish = searchAndCount_single(bigram_list, "economic", EconomicActivity_Dovish)
print(ea_dovish)

ru_hawkish = searchAndCount_single(bigram_list, "resource", ResourceUtilization_Hawkish)
print(ru_hawkish)
ru_dovish = searchAndCount_single(bigram_list, "resource", ResourceUtilization_Dovish)
print(ru_dovish)

e_hawkish = searchAndCount_single(bigram_list, "employment", Employment_Hawkish)
print(e_hawkish)
e_dovish = searchAndCount_single(bigram_list, "employment", Employment_Dovish)
print(e_dovish)

lm_hawkish = searchAndCount_single(bigram_list, "labour", LaborMarket_Hawkish)
print(lm_hawkish)
lm_dovish = searchAndCount_single(bigram_list, "labour", LaborMarket_Dovish)
print(lm_dovish)

ue_hawkish = searchAndCount_single(bigram_list, "unemployment", Unemployment_Hawkish)
print(ue_hawkish)
ue_dovish = searchAndCount_single(bigram_list, "unemployment", Unemployment_Dovish)
print(ue_dovish)

0
0
100
55
0
0
0
0
0
0
0
1
0
0
0
0


In [249]:
# Calculating net hawkishness

hawk = cpi_hawkish + ip_hawkish + cs_hawkish + ea_hawkish + ru_hawkish + e_hawkish + lm_hawkish + ue_hawkish
dove = cpi_dovish + ip_dovish + cs_dovish + ea_dovish + ru_dovish + e_dovish + lm_dovish + ue_dovish

net_hawkishness = 1 + ((hawk - dove) / (hawk + dove))

print(net_hawkishness)

1.282051282051282


In [250]:
test1 = searchAndCount_single(bigram_list, "cpi", ConsumerPricesInflation_Hawkish)
test2 = searchAndCount_double(bigram_list, "cpi", "inflation", ConsumerPricesInflation_Dovish)
test3 = searchAndCount_triple(trigram_list, "consumer", "prices", "inflation", ConsumerPricesInflation_Hawkish)

In [251]:
print(test1, test2, test3)

11 5 0


In [252]:
# there is  a file named 'Governer Speeches' which contain all the folders with speeches in them. I want to open the files on by one and run the code on them.
# after running the code I want to make dictionary of the net hawkishness of each speech and the date of the speech. and save it as a json file with the same name as the file name i run the code
# go to the folder 'Governer Speeches' and for all the subfolders in it open eaach pdf file in it and run the code

# opening each folder
os.chdir('/home/uttkarsh/IP')

def get_folder_names():
    cwd = os.getcwd()
    print(cwd)
    folder_names = glob.glob(cwd + '/Governor Speeches/*')
    return folder_names

# opening each file in each folder
def make_wordcloud_image():
    folder_names = get_folder_names()
    print(folder_names)
    # show this as tqdm
    for i in tqdm(range(len(folder_names))):
        os.chdir(folder_names[i])
        file_names = glob.glob('*.pdf')
        for file_name in file_names:
            text = get_text(file_name)
            file_name = file_name[:-4] # removing .pdf from the file name
            if os.path.exists(file_name + '_trigram' + 'json'):
                os.remove(file_name + '_trigram' + 'json')
            if os.path.exists(file_name + 'json'):
                os.remove(file_name + 'json')
            if os.path.exists(file_name + '.json'):
                os.remove(file_name + '.json')
            token = filter(text)
            string_list = get_string_from_list(token)
            bigram_list = get_bigram(token)
            trigram_list = get_trigram(token)
            wordcloud = WordCloud(width=1600, height=800, max_font_size=200).generate(string_list)
            wordcloud.to_file(file_name + '.png')
            with open(file_name + '_bigram' + '.json', 'w') as fp:
                json.dump(bigram_list, fp, indent=4)
            with open(file_name + '_trigram' + '.json', 'w') as fp:
                json.dump(trigram_list, fp, indent=4)
    return None

In [253]:
make_wordcloud_image()

/home/uttkarsh/IP
['/home/uttkarsh/IP/Governor Speeches/2021 Q1', '/home/uttkarsh/IP/Governor Speeches/2018 Q1', '/home/uttkarsh/IP/Governor Speeches/2014 Q1', '/home/uttkarsh/IP/Governor Speeches/2019 Q3', '/home/uttkarsh/IP/Governor Speeches/2014 Q2', '/home/uttkarsh/IP/Governor Speeches/2017 Q4', '/home/uttkarsh/IP/Governor Speeches/2023 Q2', '/home/uttkarsh/IP/Governor Speeches/2022 Q3', '/home/uttkarsh/IP/Governor Speeches/2016 Q1', '/home/uttkarsh/IP/Governor Speeches/2023 Q1', '/home/uttkarsh/IP/Governor Speeches/2016 Q2', '/home/uttkarsh/IP/Governor Speeches/2016 Q3', '/home/uttkarsh/IP/Governor Speeches/2022 Q1', '/home/uttkarsh/IP/Governor Speeches/2018 Q4', '/home/uttkarsh/IP/Governor Speeches/2015 Q2', '/home/uttkarsh/IP/Governor Speeches/2021 Q3', '/home/uttkarsh/IP/Governor Speeches/2023 Q3', '/home/uttkarsh/IP/Governor Speeches/2020 Q1', '/home/uttkarsh/IP/Governor Speeches/2019 Q2', '/home/uttkarsh/IP/Governor Speeches/2020 Q3']


100%|██████████| 20/20 [01:27<00:00,  4.37s/it]
